# Helical Torsion Spring

<table><tr>
<td>
<img src="./img/Helical_Torsion_Spring/Torsion_Spring.png" width=330"/> </td>
<td>
<img src="./img/Helical_Torsion_Spring/Torsion_Spring_-_End_Types.png" width=200"/> </td>
</tr></table>

Importing the relevant Classes

In [1]:
from me_toolbox.springs import HelicalTorsionSpring, Spring
from me_toolbox.tools import Nmm_per_rad_to_Nmm_per_deg

## Geometric properties

d - Wire diameter<br>
D - Spring mean diameter<br>
l1 - First leg<br>
l2 - second leg

In [2]:
d = 6  # [mm]
D = 50  # [mm]
l1 = 20  # [mm]
l2 = 20  # [mm]

## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=360"/> </td>
<td>
<img src="./img/Helical_Extension_Spring/Mechanical_properties_of_some_spring_wires_-_table_10-5.png" width=300"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

G - Shear modulus<br>
E - Elastic modulus<br>
Sut - Ultimate tensile strength

In [3]:
# music wire
G = 75e3  # [MPa]
E = 205e3  # [MPa]
Sut = Spring.material_prop('music wire', d, metric=True, verbose=False)
print(f"Sut = {Sut:.2f}")

Sut = 1705.13


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/present_of_tensile_strength.png" width=600"/> </td>
</tr>
</table>

><b>Note: </b>For Torsion springs we only use the values from the first column devided by <b>0.577</b>


In [4]:
yield_percent = 0.45 / 0.577
print(f"yield_percent = {yield_percent:.2}")

yield_percent = 0.78


## Forces and spring rate

k - Spring rate<br>
Tmax - Maximum Torque<br>
Tmin - Minimum Torque

In [5]:
k = 6  # [Nmm/rad]
Tmax = 500  # [Nm]
Tmin = 100  # [Nm]

><b>Note:</b> For torsion spring the spring rate is $\Large\frac{N \cdot mm}{rad}$
and that is what our class uses but some times the industry uses $\Large\frac{N \cdot mm}{deg}$
so I added a conversion function

In [6]:
print(f"k={Nmm_per_rad_to_Nmm_per_deg(k):.4f} [Nmm/deg]")

k=0.1047 [Nmm/deg]


## Defining the spring

Required parameters:
  * max_moment - The maximum torque in $[Nmm]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * leg1 - On of the spring's legs $[mm]$
  * leg2 - The second spring's legs $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * end_type - The shape of the ends of the spring (used to calculate number of coils)
  * spring_rate - The spring rate in $[Nmm/rad]$

Optional parameters: <br>
(the bold parameters are the defaults)

  * radius=**None** - The distance from the center to the applied force $[mm]$
  * arbor_diameter=**None** - The arbor size (Torsion springs usually used with a rod for support) $[mm]$
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$

In [7]:
spring = HelicalTorsionSpring(max_moment=Tmax,
                              wire_diameter=d,
                              spring_diameter=D,
                              leg1=l1,
                              leg2=l2,
                              ultimate_tensile_strength=Sut,
                              yield_percent=yield_percent,
                              shear_modulus=G,
                              elastic_modulus=E,
                              spring_rate=k,
                              radius=None,
                              arbor_diameter=None,
                              shot_peened=True,
                              density=None)

### Calculated geometric properties

In [8]:
print(f"Free Length (L) = {spring.length:.2f}\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Total Coils (Nb) = {spring.body_coils:.2f}")

Free Length (L) = 78303.72
Active Coils (Na) = 13050.71
Total Coils (Nb) = 13050.62


### Calculated material properties

In [9]:
print(f"Shear yield strength (Sy) = {spring.yield_strength:.2f}")

Shear yield strength (Sy) = 1329.82


## Static analysis

In [10]:
print(f"static safety factor = {spring.static_safety_factor():.2f}")

static safety factor = 51.36


## Fatigue analysis

In [11]:
nf, ns = spring.fatigue_analysis(max_moment=Tmax, min_moment=Tmin, reliability=99.999, verbose=True)
print(f"Fatigue safety factor (nf) = {nf:.2f}\n"
      f"Static safety factor (ns) = {ns:.2f}")

Alternating moment = 200.0, Mean moment = 300.0
Alternating stress = 10.35739643124631, Mean stress = 15.536094646869463
Sse = 335.60661655625483, Se= 581.6405832864036
Fatigue safety factor (nf) = 46.20
Static safety factor (ns) = 51.36


## Frequency analysis

TBD